# Large-Scale Stochastic Variational GP Regression (CUDA) (w/ SVGP)

## Overview

In this notebook, we'll give an overview of how to use Deep Kernel Learning with SVGP stochastic variational regression to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. 

In contrast to the SVDKL_Regression_GridInterp_CUDA notebook, we'll be using SVGP (https://arxiv.org/pdf/1411.2005.pdf) here to learn the inducing point locations. Our implementation of SVGP is modified to be efficient with the inference techniques used in GPyTorch.

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `song` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

## Defining the DKL Feature Extractor

Next, we define the neural network feature extractor used to define the deep kernel. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [4]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('bn1', torch.nn.BatchNorm1d(1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 1000))
        self.add_module('bn2', torch.nn.BatchNorm1d(1000))
        self.add_module('relu2', torch.nn.ReLU())                       
        self.add_module('linear3', torch.nn.Linear(1000, 500))
        self.add_module('bn3', torch.nn.BatchNorm1d(500))
        self.add_module('relu3', torch.nn.ReLU())                  
        self.add_module('linear4', torch.nn.Linear(500, 50))       
        self.add_module('bn4', torch.nn.BatchNorm1d(50))
        self.add_module('relu4', torch.nn.ReLU())                  
        self.add_module('linear5', torch.nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()
# num_features is the number of final features extracted by the neural network, in this case 2.
num_features = 2

## Defining the GP Regression Layer

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. In this example, because we will be learning the inducing point locations, we'll be using a base `VariationalStrategy` with `learn_inducing_locations=True`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

In [5]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
class GPRegressionLayer(AbstractVariationalGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPRegressionLayer, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(
            log_lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(0.001, 1., sigma=0.1, log_transform=True)
        ))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Defining the DKL Model

With the feature extractor and GP regression layer defined, we can now define our full model. To do this, we simply create a module whose `forward()` method passes the data first through the feature extractor, and then through the GP regression layer.

The only other interesting feature of the model below is that we use a helper function, `scale_to_bounds`, to ensure that the features extracted by the neural network fit within the grid bounds used for SKI.

In [6]:
class DKLModel(gpytorch.Module):
    def __init__(self, inducing_points, feature_extractor, num_features, grid_bounds=(-1., 1.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GPRegressionLayer(inducing_points)
        self.grid_bounds = grid_bounds
        self.num_features = num_features

    def forward(self, x):
        features = self.feature_extractor(x)
        features = gpytorch.utils.grid.scale_to_bounds(features, self.grid_bounds[0], self.grid_bounds[1])
        res = self.gp_layer(features)
        return res
inducing_points = gpytorch.utils.grid.scale_to_bounds(feature_extractor(train_x[:500, :]), -1, 1)
model = DKLModel(inducing_points=inducing_points, feature_extractor=feature_extractor, num_features=num_features).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

## Training the Model

The cell below trains the DKL model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalMarginalLogLikelihood` or ELBO), call backwards, and do a step of optimization.

In [7]:
model.train()
likelihood.train()

# We'll do 1 epochs of training in this tutorial
num_epochs = 6

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-3},
    {'params': model.gp_layer.parameters()},
    {'params': likelihood.parameters()},
], lr=0.1)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=train_y.size(0), combine_terms=False)

for i in range(num_epochs):
    scheduler.step()
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):        
        optimizer.zero_grad()
        
        # Because the grid is relatively small, we turn off the Toeplitz matrix multiplication and just perform them directly
        # We find this to be more efficient when the grid is very small.
        with gpytorch.settings.use_toeplitz(False):
            output = model(x_batch)
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

        # The actual optimization step
        loss.backward()
        optimizer.step()

Epoch 1 [0/340] - Loss: 188.699 [-188.699, 0.000, -0.000]
Epoch 1 [1/340] - Loss: 172.235 [-171.928, 0.308, -0.000]
Epoch 1 [2/340] - Loss: 153.265 [-152.801, 0.464, -0.000]
Epoch 1 [3/340] - Loss: 124.489 [-123.715, 0.774, -0.000]
Epoch 1 [4/340] - Loss: 105.279 [-104.167, 1.112, -0.000]
Epoch 1 [5/340] - Loss: 109.016 [-107.591, 1.426, -0.000]
Epoch 1 [6/340] - Loss: 96.918 [-95.224, 1.695, -0.000]
Epoch 1 [7/340] - Loss: 89.294 [-87.394, 1.899, -0.000]
Epoch 1 [8/340] - Loss: 83.220 [-81.187, 2.033, -0.000]
Epoch 1 [9/340] - Loss: 78.931 [-76.814, 2.117, -0.000]
Epoch 1 [10/340] - Loss: 69.420 [-67.265, 2.155, -0.000]
Epoch 1 [11/340] - Loss: 63.775 [-61.620, 2.155, -0.000]
Epoch 1 [12/340] - Loss: 69.921 [-67.785, 2.136, -0.000]
Epoch 1 [13/340] - Loss: 59.867 [-57.759, 2.108, -0.000]
Epoch 1 [14/340] - Loss: 55.385 [-53.325, 2.060, -0.000]
Epoch 1 [15/340] - Loss: 48.809 [-46.826, 1.983, -0.000]
Epoch 1 [16/340] - Loss: 43.310 [-41.422, 1.888, -0.000]
Epoch 1 [17/340] - Loss: 41.8

Epoch 1 [147/340] - Loss: 5.302 [-5.281, 0.021, -0.000]
Epoch 1 [148/340] - Loss: 6.141 [-6.122, 0.018, -0.000]
Epoch 1 [149/340] - Loss: 5.242 [-5.225, 0.017, -0.000]
Epoch 1 [150/340] - Loss: 5.312 [-5.295, 0.017, -0.000]
Epoch 1 [151/340] - Loss: 5.065 [-5.048, 0.017, -0.000]
Epoch 1 [152/340] - Loss: 5.481 [-5.464, 0.017, -0.000]
Epoch 1 [153/340] - Loss: 5.616 [-5.599, 0.017, -0.000]
Epoch 1 [154/340] - Loss: 5.182 [-5.164, 0.017, -0.000]
Epoch 1 [155/340] - Loss: 5.043 [-5.026, 0.017, -0.000]
Epoch 1 [156/340] - Loss: 5.924 [-5.907, 0.017, -0.000]
Epoch 1 [157/340] - Loss: 5.781 [-5.766, 0.015, -0.000]
Epoch 1 [158/340] - Loss: 5.499 [-5.484, 0.015, -0.000]
Epoch 1 [159/340] - Loss: 5.674 [-5.660, 0.014, -0.000]
Epoch 1 [160/340] - Loss: 7.214 [-7.201, 0.013, -0.000]
Epoch 1 [161/340] - Loss: 6.009 [-5.996, 0.012, -0.000]
Epoch 1 [162/340] - Loss: 5.632 [-5.620, 0.011, -0.000]
Epoch 1 [163/340] - Loss: 5.539 [-5.528, 0.011, -0.000]
Epoch 1 [164/340] - Loss: 5.411 [-5.400, 0.011, 

Epoch 1 [295/340] - Loss: 4.217 [-4.207, 0.011, -0.000]
Epoch 1 [296/340] - Loss: 4.352 [-4.342, 0.011, -0.000]
Epoch 1 [297/340] - Loss: 4.324 [-4.313, 0.010, -0.000]
Epoch 1 [298/340] - Loss: 4.221 [-4.211, 0.010, -0.000]
Epoch 1 [299/340] - Loss: 4.274 [-4.264, 0.010, -0.000]
Epoch 1 [300/340] - Loss: 4.317 [-4.307, 0.009, -0.000]
Epoch 1 [301/340] - Loss: 4.198 [-4.188, 0.009, -0.000]
Epoch 1 [302/340] - Loss: 4.122 [-4.113, 0.009, -0.000]
Epoch 1 [303/340] - Loss: 4.119 [-4.110, 0.009, -0.000]
Epoch 1 [304/340] - Loss: 4.179 [-4.170, 0.009, -0.000]
Epoch 1 [305/340] - Loss: 4.213 [-4.205, 0.008, -0.000]
Epoch 1 [306/340] - Loss: 4.230 [-4.222, 0.008, -0.000]
Epoch 1 [307/340] - Loss: 4.150 [-4.141, 0.008, -0.000]
Epoch 1 [308/340] - Loss: 4.222 [-4.213, 0.008, -0.000]
Epoch 1 [309/340] - Loss: 4.219 [-4.211, 0.009, -0.000]
Epoch 1 [310/340] - Loss: 4.115 [-4.107, 0.009, -0.000]
Epoch 1 [311/340] - Loss: 4.185 [-4.176, 0.009, -0.000]
Epoch 1 [312/340] - Loss: 4.135 [-4.126, 0.009, 

Epoch 2 [105/340] - Loss: 4.050 [-4.041, 0.009, -0.000]
Epoch 2 [106/340] - Loss: 4.066 [-4.058, 0.008, -0.000]
Epoch 2 [107/340] - Loss: 4.174 [-4.165, 0.008, -0.000]
Epoch 2 [108/340] - Loss: 4.149 [-4.141, 0.008, -0.000]
Epoch 2 [109/340] - Loss: 4.095 [-4.087, 0.009, -0.000]
Epoch 2 [110/340] - Loss: 3.992 [-3.983, 0.009, -0.000]
Epoch 2 [111/340] - Loss: 4.182 [-4.173, 0.009, -0.000]
Epoch 2 [112/340] - Loss: 4.079 [-4.069, 0.009, -0.000]
Epoch 2 [113/340] - Loss: 4.056 [-4.047, 0.009, -0.000]
Epoch 2 [114/340] - Loss: 4.200 [-4.192, 0.009, -0.000]
Epoch 2 [115/340] - Loss: 3.974 [-3.965, 0.009, -0.000]
Epoch 2 [116/340] - Loss: 4.180 [-4.171, 0.009, -0.000]
Epoch 2 [117/340] - Loss: 4.110 [-4.101, 0.009, -0.000]
Epoch 2 [118/340] - Loss: 3.956 [-3.947, 0.009, -0.000]
Epoch 2 [119/340] - Loss: 4.125 [-4.116, 0.009, -0.000]
Epoch 2 [120/340] - Loss: 4.037 [-4.028, 0.009, -0.000]
Epoch 2 [121/340] - Loss: 4.103 [-4.094, 0.009, -0.000]
Epoch 2 [122/340] - Loss: 4.161 [-4.153, 0.008, 

Epoch 2 [255/340] - Loss: 4.051 [-4.045, 0.007, -0.000]
Epoch 2 [256/340] - Loss: 4.124 [-4.118, 0.006, -0.000]
Epoch 2 [257/340] - Loss: 4.033 [-4.027, 0.006, -0.000]
Epoch 2 [258/340] - Loss: 3.994 [-3.989, 0.005, -0.000]
Epoch 2 [259/340] - Loss: 4.005 [-3.999, 0.005, -0.000]
Epoch 2 [260/340] - Loss: 4.025 [-4.020, 0.005, -0.000]
Epoch 2 [261/340] - Loss: 4.073 [-4.067, 0.005, -0.000]
Epoch 2 [262/340] - Loss: 4.095 [-4.090, 0.005, -0.000]
Epoch 2 [263/340] - Loss: 4.086 [-4.081, 0.005, -0.000]
Epoch 2 [264/340] - Loss: 4.063 [-4.059, 0.005, -0.000]
Epoch 2 [265/340] - Loss: 4.028 [-4.023, 0.005, -0.000]
Epoch 2 [266/340] - Loss: 4.075 [-4.070, 0.005, -0.000]
Epoch 2 [267/340] - Loss: 4.082 [-4.077, 0.005, -0.000]
Epoch 2 [268/340] - Loss: 4.003 [-3.999, 0.005, -0.000]
Epoch 2 [269/340] - Loss: 4.120 [-4.115, 0.005, -0.000]
Epoch 2 [270/340] - Loss: 4.037 [-4.032, 0.005, -0.000]
Epoch 2 [271/340] - Loss: 3.984 [-3.979, 0.005, -0.000]
Epoch 2 [272/340] - Loss: 4.075 [-4.070, 0.005, 

...

Epoch 4 [320/340] - Loss: 3.952 [-3.950, 0.002, -0.000]
Epoch 4 [321/340] - Loss: 3.972 [-3.969, 0.002, -0.000]
Epoch 4 [322/340] - Loss: 3.998 [-3.996, 0.002, -0.000]
Epoch 4 [323/340] - Loss: 3.969 [-3.967, 0.002, -0.000]
Epoch 4 [324/340] - Loss: 3.990 [-3.988, 0.002, -0.000]
Epoch 4 [325/340] - Loss: 3.983 [-3.980, 0.002, -0.000]
Epoch 4 [326/340] - Loss: 3.961 [-3.959, 0.002, -0.000]
Epoch 4 [327/340] - Loss: 4.068 [-4.066, 0.002, -0.000]
Epoch 4 [328/340] - Loss: 4.021 [-4.019, 0.002, -0.000]
Epoch 4 [329/340] - Loss: 4.023 [-4.020, 0.002, -0.000]
Epoch 4 [330/340] - Loss: 3.980 [-3.978, 0.002, -0.000]
Epoch 4 [331/340] - Loss: 4.005 [-4.003, 0.002, -0.000]
Epoch 4 [332/340] - Loss: 3.982 [-3.980, 0.002, -0.000]
Epoch 4 [333/340] - Loss: 3.954 [-3.952, 0.002, -0.000]
Epoch 4 [334/340] - Loss: 4.007 [-4.005, 0.002, -0.000]
Epoch 4 [335/340] - Loss: 3.944 [-3.941, 0.002, -0.000]
Epoch 4 [336/340] - Loss: 3.968 [-3.965, 0.002, -0.000]
Epoch 4 [337/340] - Loss: 3.967 [-3.965, 0.002, 

Epoch 5 [129/340] - Loss: 3.936 [-3.934, 0.002, -0.000]
Epoch 5 [130/340] - Loss: 3.993 [-3.991, 0.002, -0.000]
Epoch 5 [131/340] - Loss: 3.933 [-3.931, 0.002, -0.000]
Epoch 5 [132/340] - Loss: 3.951 [-3.948, 0.002, -0.000]
Epoch 5 [133/340] - Loss: 3.900 [-3.898, 0.002, -0.000]
Epoch 5 [134/340] - Loss: 3.932 [-3.930, 0.002, -0.000]
Epoch 5 [135/340] - Loss: 3.953 [-3.951, 0.002, -0.000]
Epoch 5 [136/340] - Loss: 3.901 [-3.899, 0.002, -0.000]
Epoch 5 [137/340] - Loss: 3.917 [-3.915, 0.002, -0.000]
Epoch 5 [138/340] - Loss: 3.925 [-3.923, 0.002, -0.000]
Epoch 5 [139/340] - Loss: 3.985 [-3.983, 0.002, -0.000]
Epoch 5 [140/340] - Loss: 3.933 [-3.930, 0.002, -0.000]
Epoch 5 [141/340] - Loss: 3.931 [-3.929, 0.002, -0.000]
Epoch 5 [142/340] - Loss: 3.921 [-3.918, 0.002, -0.000]
Epoch 5 [143/340] - Loss: 3.987 [-3.985, 0.002, -0.000]
Epoch 5 [144/340] - Loss: 3.972 [-3.970, 0.002, -0.000]
Epoch 5 [145/340] - Loss: 4.013 [-4.011, 0.002, -0.000]
Epoch 5 [146/340] - Loss: 3.946 [-3.944, 0.002, 

Epoch 5 [277/340] - Loss: 3.914 [-3.912, 0.002, -0.000]
Epoch 5 [278/340] - Loss: 3.931 [-3.929, 0.002, -0.000]
Epoch 5 [279/340] - Loss: 4.020 [-4.018, 0.002, -0.000]
Epoch 5 [280/340] - Loss: 3.958 [-3.955, 0.002, -0.000]
Epoch 5 [281/340] - Loss: 3.969 [-3.967, 0.002, -0.000]
Epoch 5 [282/340] - Loss: 3.972 [-3.970, 0.002, -0.000]
Epoch 5 [283/340] - Loss: 3.949 [-3.947, 0.002, -0.000]
Epoch 5 [284/340] - Loss: 3.966 [-3.964, 0.002, -0.000]
Epoch 5 [285/340] - Loss: 3.927 [-3.925, 0.002, -0.000]
Epoch 5 [286/340] - Loss: 3.982 [-3.980, 0.002, -0.000]
Epoch 5 [287/340] - Loss: 4.041 [-4.039, 0.002, -0.000]
Epoch 5 [288/340] - Loss: 3.957 [-3.955, 0.002, -0.000]
Epoch 5 [289/340] - Loss: 3.956 [-3.954, 0.002, -0.000]
Epoch 5 [290/340] - Loss: 3.934 [-3.932, 0.002, -0.000]
Epoch 5 [291/340] - Loss: 3.979 [-3.976, 0.002, -0.000]
Epoch 5 [292/340] - Loss: 3.922 [-3.920, 0.002, -0.000]
Epoch 5 [293/340] - Loss: 3.918 [-3.916, 0.002, -0.000]
Epoch 5 [294/340] - Loss: 3.936 [-3.934, 0.002, 

Epoch 6 [85/340] - Loss: 3.886 [-3.884, 0.002, -0.000]
Epoch 6 [86/340] - Loss: 3.891 [-3.889, 0.002, -0.000]
Epoch 6 [87/340] - Loss: 3.928 [-3.926, 0.002, -0.000]
Epoch 6 [88/340] - Loss: 3.918 [-3.916, 0.002, -0.000]
Epoch 6 [89/340] - Loss: 4.003 [-4.001, 0.002, -0.000]
Epoch 6 [90/340] - Loss: 3.937 [-3.935, 0.002, -0.000]
Epoch 6 [91/340] - Loss: 3.896 [-3.894, 0.002, -0.000]
Epoch 6 [92/340] - Loss: 3.963 [-3.960, 0.002, -0.000]
Epoch 6 [93/340] - Loss: 3.924 [-3.922, 0.002, -0.000]
Epoch 6 [94/340] - Loss: 3.954 [-3.952, 0.002, -0.000]
Epoch 6 [95/340] - Loss: 3.935 [-3.933, 0.002, -0.000]
Epoch 6 [96/340] - Loss: 3.891 [-3.889, 0.002, -0.000]
Epoch 6 [97/340] - Loss: 3.942 [-3.940, 0.002, -0.000]
Epoch 6 [98/340] - Loss: 4.017 [-4.014, 0.002, -0.000]
Epoch 6 [99/340] - Loss: 3.964 [-3.962, 0.002, -0.000]
Epoch 6 [100/340] - Loss: 3.963 [-3.961, 0.002, -0.000]
Epoch 6 [101/340] - Loss: 3.892 [-3.890, 0.002, -0.000]
Epoch 6 [102/340] - Loss: 3.889 [-3.886, 0.002, -0.000]
Epoch 6

Epoch 6 [232/340] - Loss: 3.895 [-3.893, 0.002, -0.000]
Epoch 6 [233/340] - Loss: 3.950 [-3.948, 0.002, -0.000]
Epoch 6 [234/340] - Loss: 3.874 [-3.871, 0.002, -0.000]
Epoch 6 [235/340] - Loss: 3.900 [-3.897, 0.002, -0.000]
Epoch 6 [236/340] - Loss: 3.964 [-3.961, 0.002, -0.000]
Epoch 6 [237/340] - Loss: 3.851 [-3.849, 0.002, -0.000]
Epoch 6 [238/340] - Loss: 3.900 [-3.898, 0.002, -0.000]
Epoch 6 [239/340] - Loss: 3.940 [-3.937, 0.002, -0.000]
Epoch 6 [240/340] - Loss: 3.956 [-3.953, 0.002, -0.000]
Epoch 6 [241/340] - Loss: 3.894 [-3.891, 0.002, -0.000]
Epoch 6 [242/340] - Loss: 3.951 [-3.949, 0.002, -0.000]
Epoch 6 [243/340] - Loss: 3.902 [-3.899, 0.002, -0.000]
Epoch 6 [244/340] - Loss: 3.934 [-3.931, 0.002, -0.000]
Epoch 6 [245/340] - Loss: 3.913 [-3.911, 0.002, -0.000]
Epoch 6 [246/340] - Loss: 3.926 [-3.924, 0.002, -0.000]
Epoch 6 [247/340] - Loss: 3.905 [-3.903, 0.002, -0.000]
Epoch 6 [248/340] - Loss: 3.914 [-3.912, 0.002, -0.000]
Epoch 6 [249/340] - Loss: 3.875 [-3.872, 0.002, 

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although our other tutorials demonstrate how to do this (for example, see the CIFAR tutorial).

In [8]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False):
    preds = model(test_x)

In [9]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 8.560413360595703
